In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# FastAPI for serving risk analytics
from fastapi import FastAPI
from pydantic import BaseModel

# Placeholder for ORE bindings (replace with actual ORE SWIG imports)
# from ore import OREAnalytics


In [ ]:
# Equity quotes
quotes_equity = pd.DataFrame(columns=[
    "symbol","timestamp","open","high","low","close","volume","dividend"
])

# Options chain
options_chain = pd.DataFrame(columns=[
    "symbol","expiry","strike","type","timestamp","bid","ask","iv","oi"
])


In [ ]:
class OREAnalyticsStub:
    def price(self, instrument, market):
        return {"npv": 100.0, "delta": 0.65, "gamma": 0.02, "vega": 0.8}
    def portfolio_var(self, positions, market, horizon_days=1, method="historical"):
        return {"VaR": 0.018, "ES": 0.026}
    def stress(self, positions, market, shocks):
        return {"shock_-5pct": -0.032, "vol_spike_20pct": -0.015}

ORE = OREAnalyticsStub()


In [ ]:
dates = pd.date_range(end=pd.Timestamp.today().normalize(), periods=90, freq="B")
price = 1500 + np.cumsum(np.random.normal(0, 5, len(dates)))
high = price + np.random.uniform(5, 15, len(dates))
low = price - np.random.uniform(5, 15, len(dates))
open_ = price + np.random.normal(0, 3, len(dates))
close = price + np.random.normal(0, 3, len(dates))
volume = np.random.randint(5e6, 12e6, len(dates))

ohlcv = pd.DataFrame({
    "timestamp": dates,
    "open": open_,
    "high": high,
    "low": low,
    "close": close,
    "volume": volume,
    "symbol": "INFY"
})
ohlcv.tail()


In [ ]:
def compute_atr(df: pd.DataFrame, period: int = 14) -> float:
    tr = pd.concat([
        (df["high"] - df["low"]).abs(),
        (df["high"] - df["close"].shift(1)).abs(),
        (df["low"] - df["close"].shift(1)).abs()
    ], axis=1).max(axis=1)
    atr = tr.rolling(period).mean().iloc[-1]
    return float(atr)

def entry_band(price: float, width_bps: int = 50):
    w = price * width_bps / 10000.0
    return [round(price - w, 2), round(price + w, 2)]

def target_band(price: float, low_pct: float = 0.05, high_pct: float = 0.08):
    return [round(price * (1 + low_pct), 2), round(price * (1 + high_pct), 2)]

def size_by_var(account_equity: float, current_var: float, var_cap: float, delta: float, price: float, assumed_move: float = 0.015):
    max_inc_var = max(var_cap - current_var, 0.0)
    if max_inc_var <= 0:
        return 0
    rupee_var_per_share = delta * price * assumed_move / account_equity
    size = int(max_inc_var / max(rupee_var_per_share, 1e-9))
    return max(size, 0)


In [ ]:
def pre_trade(symbol, ts, account_equity, var_cap, quotes_df, chain_df, price_series):
    market_price = 1500  # mock price
    analytics = ORE.price({"type":"Equity","symbol":symbol}, {"price": market_price})
    portfolio_metrics = ORE.portfolio_var([], {"price": market_price}, horizon_days=1)

    # Simple sizing rule: keep VaR under cap
    size = int((var_cap - portfolio_metrics["VaR"]) * account_equity / (analytics["delta"] * market_price * 0.015))

    stop = market_price - 1.2 * price_series.std()
    target_low = market_price * 1.05
    target_high = market_price * 1.08

    return {
        "symbol": symbol,
        "price": market_price,
        "delta": analytics["delta"],
        "suggested_size_shares": size,
        "entry_band": [market_price*0.995, market_price*1.005],
        "stop_loss": round(stop, 2),
        "target_21D": [round(target_low,2), round(target_high,2)],
        "current_portfolio_var": portfolio_metrics["VaR"],
        "post_trade_var_estimate": portfolio_metrics["VaR"] + 0.002,
        "notes": ["ATR-based stop", "VaR capped at user limit"]
    }


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 8))

ax[0].plot(ohlcv["timestamp"], ohlcv["close"], label="Close", color="#2a9d8f")
ax[0].axhline(pre_trade_card["stop_loss"], color="#e76f51", linestyle="--", label="Stop")
ax[0].axhline(pre_trade_card["target_price_band_21D"][0], color="#457b9d", linestyle=":", label="Target Low")
ax[0].axhline(pre_trade_card["target_price_band_21D"][1], color="#1d3557", linestyle=":", label="Target High")
ax[0].set_title("INFY price with stop & target bands")
ax[0].legend()

var_val = pre_trade_card["current_portfolio_var"]
cap_val = user_var_cap
ax[1].bar(["Current VaR", "User Cap"], [var_val, cap_val], color=["#f4a261", "#264653"])
ax[1].set_ylim(0, max(0.05, cap_val + 0.01))
ax[1].set_title("Portfolio VaR vs. user cap")
plt.tight_layout()
plt.show()
